In [1]:
## !pip install sqlalchemy
!pip install pymysql

Installing the essential libraries

In [2]:
#Installing all the essential libraries first
import pandas as pd
import requests
from bs4 import BeautifulSoup
!pip install lat_lon_parser
from datetime import datetime

# Calling the cities function

This function consists the cities name, lat/long and population data

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse    # for decimal coordinates


def cities_dataframe(cities):

  city_data = []

  for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # extract the relevant information
    city_population = city_soup.find(string="Population").find_next("td").get_text()
    city_population_clean = int(city_population.replace(",", ""))
    today = datetime.today().strftime("%d.%m.%Y")  
    city_latitude = city_soup.find(class_="latitude").get_text()
    city_longitude = city_soup.find(class_="longitude").get_text()
    country = city_soup.find(class_="infobox-data").get_text()

    # Append the information to the empty lists
    city_data.append({"City": city,
                    "Country": country,
                    "Latitude": parse(city_latitude), # latitude in decimal format
                    "Longitude": parse(city_longitude), # longitude in decimal format
                    "Population": city_population_clean,
                    "Timestamp": today
                    })

  return pd.DataFrame(city_data)

Calling the function

In [4]:
# call the function
list_of_cities = ["Berlin", "Hamburg", "Munich"]

cities_df = cities_dataframe(list_of_cities)
cities_df

,City,Country,Latitude,Longitude,Population,Timestamp
0,Berlin,Germany,52.5200,13.405,3878100,08.07.2024
1,Hamburg,Germany,53.5500,10.000,1964021,08.07.2024
2,Munich,Germany,48.1375,11.575,1512491,08.07.2024


Dropping the duplicates and keeping only unique cities with countries (in our case we have only unique cities)

In [5]:
Cities_df = cities_df.drop_duplicates(subset='City')[['City', 'Country']].reset_index(drop=True)
Cities_df

,City,Country
0,Berlin,Germany
1,Hamburg,Germany
2,Munich,Germany


## Establishing the SQL connection

In [11]:
schema = #"schema name" # Name of Schema changes
host = #"host id"
user = #"root"
password = #"your password" # Password changes
port = #port number

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## Pushing the data to city table in SQL

In [12]:
Cities_df.to_sql('city',
                  if_exists='append',
                  con=connection_string,
                  index=False)

## Retrieving the data from SQL table (Since we get the City_id column now)

In [13]:
city_from_sql = pd.read_sql("city", con = connection_string)
city_from_sql

,City_id,City,Country
0,1,Berlin,Germany
1,2,Hamburg,Germany
2,3,Munich,Germany


creating copy of the sql table to keep original DF untouched

In [14]:
city_from_sql1 = city_from_sql.copy()
city_from_sql1

,City_id,City,Country
0,1,Berlin,Germany
1,2,Hamburg,Germany
2,3,Munich,Germany


Merging the sql cities table with the initial table to get the City_id in the intial table

In [15]:
cities_df = cities_df.merge(city_from_sql, on = "City", how = "left")

cities_df

,City,Country_x,Latitude,Longitude,Population,Timestamp,City_id,Country_y
0,Berlin,Germany,52.5200,13.405,3878100,08.07.2024,1,Germany
1,Hamburg,Germany,53.5500,10.000,1964021,08.07.2024,2,Germany
2,Munich,Germany,48.1375,11.575,1512491,08.07.2024,3,Germany


Dropping unnecessary columns

In [16]:
cities_df = cities_df.drop(columns = ["City", "Country_x", "Country_y"])

cities_df

,Latitude,Longitude,Population,Timestamp,City_id
0,52.5200,13.405,3878100,08.07.2024,1
1,53.5500,10.000,1964021,08.07.2024,2
2,48.1375,11.575,1512491,08.07.2024,3


Converting the Timestamp column data-type to date type

In [17]:
cities_df['Timestamp'] = pd.to_datetime(cities_df['Timestamp'])
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Latitude    3 non-null      float64       
 1   Longitude   3 non-null      float64       
 2   Population  3 non-null      int64         
 3   Timestamp   3 non-null      datetime64[ns]
 4   City_id     3 non-null      int64         
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 144.0 bytes


In [18]:
cities_df.rename(columns={"Timestamp": "Timestamp_"}, inplace=True)

cities_df

,Latitude,Longitude,Population,Timestamp_,City_id
0,52.5200,13.405,3878100,2024-08-07,1
1,53.5500,10.000,1964021,2024-08-07,2
2,48.1375,11.575,1512491,2024-08-07,3


In [19]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Latitude    3 non-null      float64       
 1   Longitude   3 non-null      float64       
 2   Population  3 non-null      int64         
 3   Timestamp_  3 non-null      datetime64[ns]
 4   City_id     3 non-null      int64         
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 144.0 bytes


## Pushing the data to city_info table in SQL

In [20]:
cities_df.to_sql('city_info',
                if_exists='append',
                con=connection_string,
                index=False)

## Retrieving the data from SQL table (just to review)

In [21]:
city_info_from_sql = pd.read_sql("city_info", con = connection_string)
city_info_from_sql

,Latitude,Longitude,Population,Timestamp_,City_id
0,52.5200,13.405,3878100,2024-08-07,1
1,53.5500,10.000,1964021,2024-08-07,2
2,48.1375,11.575,1512491,2024-08-07,3


In [22]:
city_info_from_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Latitude    3 non-null      float64       
 1   Longitude   3 non-null      float64       
 2   Population  3 non-null      int64         
 3   Timestamp_  3 non-null      datetime64[ns]
 4   City_id     3 non-null      int64         
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 248.0 bytes


# Calling the weather (for multiple cities) function

In [23]:
def fetch_weather_data(cities): # Defining the function with parameter cities
    # Ensuring the timezone is local
    berlin_timezone = timezone("Europe/Berlin")
    API_key = #"Your_API_Key"
    weather_items = [] # Created an empty list which will be appended with the output from the for loop
    
    for city in cities:
        country = "DE"
        
        # Geocoder api to get the latitude and longitude
        geo_url = (f"http://api.openweathermap.org/geo/1.0/direct?q={city},{country}&limit=5&appid={API_key}")
        geo_response = requests.get(geo_url)
        geo_json = geo_response.json()
        latitude = geo_json[0]["lat"]
        longitude = geo_json[0]["lon"]
        
        # Calling the weather api
        weather_url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")
        weather_response = requests.get(weather_url)
        w_json = weather_response.json()
        
        # Added the time retrieved so we know when the forecast was made
        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

        for item in w_json["list"]:
            weather_item = {
            # Added the city name, so the information is clear when looking at multiple cities
                "city": city,
                "forecast_time": item.get("dt_txt", None),
                "temperature": item["main"].get("temp", None),
                "forecast": item["weather"][0].get("main", None),
                "forecast desc.": item["weather"][0].get("description", None),
                "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
                "wind_speed": item["wind"].get("speed", None),
                "data_retrieved_at": retrieval_time
            }

            weather_items.append(weather_item)

    weather_df = pd.DataFrame(weather_items)

    return weather_df

Importing essential libraries

In [24]:
from datetime import datetime
from pytz import timezone

Calling the function

In [25]:
fetch_weather_data(["Berlin", "Hamburg", "Munich"])

,city,forecast_time,temperature,forecast,forecast desc.,rain_in_last_3h,wind_speed,data_retrieved_at
0,Berlin,2024-07-08 12:00:00,25.44,Clear,clear sky,0.00,3.55,2024-07-08 11:59:40
1,Berlin,2024-07-08 15:00:00,25.04,Clouds,broken clouds,0.00,3.85,2024-07-08 11:59:40
2,Berlin,2024-07-08 18:00:00,23.66,Clouds,overcast clouds,0.00,2.49,2024-07-08 11:59:40
3,Berlin,2024-07-08 21:00:00,20.44,Clouds,overcast clouds,0.00,2.28,2024-07-08 11:59:40
4,Berlin,2024-07-09 00:00:00,18.36,Clouds,overcast clouds,0.00,2.07,2024-07-08 11:59:40
...,...,...,...,...,...,...,...,...
115,Munich,2024-07-12 21:00:00,16.52,Rain,light rain,0.32,2.54,2024-07-08 11:59:41
116,Munich,2024-07-13 00:00:00,16.85,Clouds,overcast clouds,0.00,2.22,2024-07-08 11:59:41
117,Munich,2024-07-13 03:00:00,16.77,Rain,light rain,0.36,2.10,2024-07-08 11:59:41
118,Munich,2024-07-13 06:00:00,16.68,Rain,light rain,2.40,1.99,2024-07-08 11:59:41


Converting to a Dataframe (DF)

In [27]:
weather_info = fetch_weather_data(["Berlin", "Hamburg", "Munich"])
weatherinfo_df = pd.DataFrame(weather_info)
weatherinfo_df

,city,forecast_time,temperature,forecast,forecast desc.,rain_in_last_3h,wind_speed,data_retrieved_at
0,Berlin,2024-07-08 12:00:00,25.44,Clear,clear sky,0.00,3.55,2024-07-08 11:59:52
1,Berlin,2024-07-08 15:00:00,25.04,Clouds,broken clouds,0.00,3.85,2024-07-08 11:59:52
2,Berlin,2024-07-08 18:00:00,23.66,Clouds,overcast clouds,0.00,2.49,2024-07-08 11:59:52
3,Berlin,2024-07-08 21:00:00,20.44,Clouds,overcast clouds,0.00,2.28,2024-07-08 11:59:52
4,Berlin,2024-07-09 00:00:00,18.36,Clouds,overcast clouds,0.00,2.07,2024-07-08 11:59:52
...,...,...,...,...,...,...,...,...
115,Munich,2024-07-12 21:00:00,16.52,Rain,light rain,0.32,2.54,2024-07-08 11:59:52
116,Munich,2024-07-13 00:00:00,16.85,Clouds,overcast clouds,0.00,2.22,2024-07-08 11:59:52
117,Munich,2024-07-13 03:00:00,16.77,Rain,light rain,0.36,2.10,2024-07-08 11:59:52
118,Munich,2024-07-13 06:00:00,16.68,Rain,light rain,2.40,1.99,2024-07-08 11:59:52


Merging the weather and city table

In [28]:
weatherinfo_merged = weatherinfo_df.merge(city_from_sql, left_on = "city", right_on = "City", how = "right")

weatherinfo_merged


,city,forecast_time,temperature,forecast,forecast desc.,rain_in_last_3h,wind_speed,data_retrieved_at,City_id,City,Country
0,Berlin,2024-07-08 12:00:00,25.44,Clear,clear sky,0.00,3.55,2024-07-08 11:59:52,1,Berlin,Germany
1,Berlin,2024-07-08 15:00:00,25.04,Clouds,broken clouds,0.00,3.85,2024-07-08 11:59:52,1,Berlin,Germany
2,Berlin,2024-07-08 18:00:00,23.66,Clouds,overcast clouds,0.00,2.49,2024-07-08 11:59:52,1,Berlin,Germany
3,Berlin,2024-07-08 21:00:00,20.44,Clouds,overcast clouds,0.00,2.28,2024-07-08 11:59:52,1,Berlin,Germany
4,Berlin,2024-07-09 00:00:00,18.36,Clouds,overcast clouds,0.00,2.07,2024-07-08 11:59:52,1,Berlin,Germany
...,...,...,...,...,...,...,...,...,...,...,...
115,Munich,2024-07-12 21:00:00,16.52,Rain,light rain,0.32,2.54,2024-07-08 11:59:52,3,Munich,Germany
116,Munich,2024-07-13 00:00:00,16.85,Clouds,overcast clouds,0.00,2.22,2024-07-08 11:59:52,3,Munich,Germany
117,Munich,2024-07-13 03:00:00,16.77,Rain,light rain,0.36,2.10,2024-07-08 11:59:52,3,Munich,Germany
118,Munich,2024-07-13 06:00:00,16.68,Rain,light rain,2.40,1.99,2024-07-08 11:59:52,3,Munich,Germany


In [29]:
weatherinfo_merged1 = weatherinfo_merged.copy()
weatherinfo_merged1 # Created a copy to keep the original untouched

,city,forecast_time,temperature,forecast,forecast desc.,rain_in_last_3h,wind_speed,data_retrieved_at,City_id,City,Country
0,Berlin,2024-07-08 12:00:00,25.44,Clear,clear sky,0.00,3.55,2024-07-08 11:59:52,1,Berlin,Germany
1,Berlin,2024-07-08 15:00:00,25.04,Clouds,broken clouds,0.00,3.85,2024-07-08 11:59:52,1,Berlin,Germany
2,Berlin,2024-07-08 18:00:00,23.66,Clouds,overcast clouds,0.00,2.49,2024-07-08 11:59:52,1,Berlin,Germany
3,Berlin,2024-07-08 21:00:00,20.44,Clouds,overcast clouds,0.00,2.28,2024-07-08 11:59:52,1,Berlin,Germany
4,Berlin,2024-07-09 00:00:00,18.36,Clouds,overcast clouds,0.00,2.07,2024-07-08 11:59:52,1,Berlin,Germany
...,...,...,...,...,...,...,...,...,...,...,...
115,Munich,2024-07-12 21:00:00,16.52,Rain,light rain,0.32,2.54,2024-07-08 11:59:52,3,Munich,Germany
116,Munich,2024-07-13 00:00:00,16.85,Clouds,overcast clouds,0.00,2.22,2024-07-08 11:59:52,3,Munich,Germany
117,Munich,2024-07-13 03:00:00,16.77,Rain,light rain,0.36,2.10,2024-07-08 11:59:52,3,Munich,Germany
118,Munich,2024-07-13 06:00:00,16.68,Rain,light rain,2.40,1.99,2024-07-08 11:59:52,3,Munich,Germany


Dropping unnecessary columns

In [30]:
weatherinfo_merged1 = weatherinfo_merged1.drop(columns = "city")
weatherinfo_merged1 

,forecast_time,temperature,forecast,forecast desc.,rain_in_last_3h,wind_speed,data_retrieved_at,City_id,City,Country
0,2024-07-08 12:00:00,25.44,Clear,clear sky,0.00,3.55,2024-07-08 11:59:52,1,Berlin,Germany
1,2024-07-08 15:00:00,25.04,Clouds,broken clouds,0.00,3.85,2024-07-08 11:59:52,1,Berlin,Germany
2,2024-07-08 18:00:00,23.66,Clouds,overcast clouds,0.00,2.49,2024-07-08 11:59:52,1,Berlin,Germany
3,2024-07-08 21:00:00,20.44,Clouds,overcast clouds,0.00,2.28,2024-07-08 11:59:52,1,Berlin,Germany
4,2024-07-09 00:00:00,18.36,Clouds,overcast clouds,0.00,2.07,2024-07-08 11:59:52,1,Berlin,Germany
...,...,...,...,...,...,...,...,...,...,...
115,2024-07-12 21:00:00,16.52,Rain,light rain,0.32,2.54,2024-07-08 11:59:52,3,Munich,Germany
116,2024-07-13 00:00:00,16.85,Clouds,overcast clouds,0.00,2.22,2024-07-08 11:59:52,3,Munich,Germany
117,2024-07-13 03:00:00,16.77,Rain,light rain,0.36,2.10,2024-07-08 11:59:52,3,Munich,Germany
118,2024-07-13 06:00:00,16.68,Rain,light rain,2.40,1.99,2024-07-08 11:59:52,3,Munich,Germany


Renaming the columns with relevant names

In [31]:
weatherinfo_merged1.rename(columns={"forecast desc.": "forecast desc_"}, inplace=True)
weatherinfo_merged1

,forecast_time,temperature,forecast,forecast desc_,rain_in_last_3h,wind_speed,data_retrieved_at,City_id,City,Country
0,2024-07-08 12:00:00,25.44,Clear,clear sky,0.00,3.55,2024-07-08 11:59:52,1,Berlin,Germany
1,2024-07-08 15:00:00,25.04,Clouds,broken clouds,0.00,3.85,2024-07-08 11:59:52,1,Berlin,Germany
2,2024-07-08 18:00:00,23.66,Clouds,overcast clouds,0.00,2.49,2024-07-08 11:59:52,1,Berlin,Germany
3,2024-07-08 21:00:00,20.44,Clouds,overcast clouds,0.00,2.28,2024-07-08 11:59:52,1,Berlin,Germany
4,2024-07-09 00:00:00,18.36,Clouds,overcast clouds,0.00,2.07,2024-07-08 11:59:52,1,Berlin,Germany
...,...,...,...,...,...,...,...,...,...,...
115,2024-07-12 21:00:00,16.52,Rain,light rain,0.32,2.54,2024-07-08 11:59:52,3,Munich,Germany
116,2024-07-13 00:00:00,16.85,Clouds,overcast clouds,0.00,2.22,2024-07-08 11:59:52,3,Munich,Germany
117,2024-07-13 03:00:00,16.77,Rain,light rain,0.36,2.10,2024-07-08 11:59:52,3,Munich,Germany
118,2024-07-13 06:00:00,16.68,Rain,light rain,2.40,1.99,2024-07-08 11:59:52,3,Munich,Germany


In [32]:
weatherinfo_merged1 = weatherinfo_merged1.drop(columns = "city")


KeyError: "['city'] not found in axis"

In [33]:
weatherinfo_merged1

,forecast_time,temperature,forecast,forecast desc_,rain_in_last_3h,wind_speed,data_retrieved_at,City_id,City,Country
0,2024-07-08 12:00:00,25.44,Clear,clear sky,0.00,3.55,2024-07-08 11:59:52,1,Berlin,Germany
1,2024-07-08 15:00:00,25.04,Clouds,broken clouds,0.00,3.85,2024-07-08 11:59:52,1,Berlin,Germany
2,2024-07-08 18:00:00,23.66,Clouds,overcast clouds,0.00,2.49,2024-07-08 11:59:52,1,Berlin,Germany
3,2024-07-08 21:00:00,20.44,Clouds,overcast clouds,0.00,2.28,2024-07-08 11:59:52,1,Berlin,Germany
4,2024-07-09 00:00:00,18.36,Clouds,overcast clouds,0.00,2.07,2024-07-08 11:59:52,1,Berlin,Germany
...,...,...,...,...,...,...,...,...,...,...
115,2024-07-12 21:00:00,16.52,Rain,light rain,0.32,2.54,2024-07-08 11:59:52,3,Munich,Germany
116,2024-07-13 00:00:00,16.85,Clouds,overcast clouds,0.00,2.22,2024-07-08 11:59:52,3,Munich,Germany
117,2024-07-13 03:00:00,16.77,Rain,light rain,0.36,2.10,2024-07-08 11:59:52,3,Munich,Germany
118,2024-07-13 06:00:00,16.68,Rain,light rain,2.40,1.99,2024-07-08 11:59:52,3,Munich,Germany


In [34]:
weatherinfo_merged1.rename(columns={"forecast desc_": "forecast_desc"}, inplace=True)
weatherinfo_merged1

,forecast_time,temperature,forecast,forecast_desc,rain_in_last_3h,wind_speed,data_retrieved_at,City_id,City,Country
0,2024-07-08 12:00:00,25.44,Clear,clear sky,0.00,3.55,2024-07-08 11:59:52,1,Berlin,Germany
1,2024-07-08 15:00:00,25.04,Clouds,broken clouds,0.00,3.85,2024-07-08 11:59:52,1,Berlin,Germany
2,2024-07-08 18:00:00,23.66,Clouds,overcast clouds,0.00,2.49,2024-07-08 11:59:52,1,Berlin,Germany
3,2024-07-08 21:00:00,20.44,Clouds,overcast clouds,0.00,2.28,2024-07-08 11:59:52,1,Berlin,Germany
4,2024-07-09 00:00:00,18.36,Clouds,overcast clouds,0.00,2.07,2024-07-08 11:59:52,1,Berlin,Germany
...,...,...,...,...,...,...,...,...,...,...
115,2024-07-12 21:00:00,16.52,Rain,light rain,0.32,2.54,2024-07-08 11:59:52,3,Munich,Germany
116,2024-07-13 00:00:00,16.85,Clouds,overcast clouds,0.00,2.22,2024-07-08 11:59:52,3,Munich,Germany
117,2024-07-13 03:00:00,16.77,Rain,light rain,0.36,2.10,2024-07-08 11:59:52,3,Munich,Germany
118,2024-07-13 06:00:00,16.68,Rain,light rain,2.40,1.99,2024-07-08 11:59:52,3,Munich,Germany


Establishing SQL connection (not reall required if the connection has been already set up)

In [35]:
schema = #"your schema" # Name of Schema changes
host = #"host"
user = #"root"
password = #"your password" # Password changes
port = # port number

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## Pushing the data to weather table in SQL

In [36]:
weatherinfo_merged1.to_sql('weather',
                  if_exists='append',
                  con=connection_string,
                  index=False)

## Retrieveing the data from SQL (just to review)

In [37]:
weather_from_sql = pd.read_sql("weather", con = connection_string)
weather_from_sql

,forecast_time,temperature,forecast,forecast_desc,rain_in_last_3h,wind_speed,data_retrieved_at,City_id,City,Country
0,2024-07-08 12:00:00,25.44,Clear,clear sky,0.00,3.55,2024-07-08 11:59:52,1,Berlin,Germany
1,2024-07-08 15:00:00,25.04,Clouds,broken clouds,0.00,3.85,2024-07-08 11:59:52,1,Berlin,Germany
2,2024-07-08 18:00:00,23.66,Clouds,overcast clouds,0.00,2.49,2024-07-08 11:59:52,1,Berlin,Germany
3,2024-07-08 21:00:00,20.44,Clouds,overcast clouds,0.00,2.28,2024-07-08 11:59:52,1,Berlin,Germany
4,2024-07-09 00:00:00,18.36,Clouds,overcast clouds,0.00,2.07,2024-07-08 11:59:52,1,Berlin,Germany
...,...,...,...,...,...,...,...,...,...,...
115,2024-07-12 21:00:00,16.52,Rain,light rain,0.32,2.54,2024-07-08 11:59:52,3,Munich,Germany
116,2024-07-13 00:00:00,16.85,Clouds,overcast clouds,0.00,2.22,2024-07-08 11:59:52,3,Munich,Germany
117,2024-07-13 03:00:00,16.77,Rain,light rain,0.36,2.10,2024-07-08 11:59:52,3,Munich,Germany
118,2024-07-13 06:00:00,16.68,Rain,light rain,2.40,1.99,2024-07-08 11:59:52,3,Munich,Germany


# Calling the airport (for multiple cities) function

Importing the essential libraries

In [38]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone

In [39]:
# Defining a function to retrieve ICAO airport codes given lists of latitudes and longitudes
def icao_airport_codes(latitudes, longitudes):
    
    # Assert len(latitudes) == len(longitudes)
    
    
    list_for_df = []  # Initializing an empty list to store dataframes
    
    for index, value in enumerate(latitudes): # Looping through the latitudes and corresponding longitudes
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{value}/{longitudes[index]}/km/50/5"
        querystring = {"withFlightInfoOnly":"true"}

        headers = {
          "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
          "X-RapidAPI-Key": #"X-RapidAPI-Key"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)
        
        # Normalizing the JSON response and append the resulting dataframe to the list
        list_for_df.append(pd.json_normalize(response.json()['items']))
        

    return pd.concat(list_for_df, ignore_index=True)  # Concatenating all dataframes in the list into a single dataframe and return it
    

In [41]:
list(enumerate(latitudes)) # Create an enumerated list of tuples from the latitudes list (index, value)

[(0, 52.52), (1, 53.5511), (2, 48.1351)]

Calling the function

In [40]:
# Entering the coordinates for Berlin, Paris, London (Desired Cities)
latitudes = [52.5200, 53.5511, 48.1351]
longitudes = [13.4050, 9.9937, 11.5820]

icao_airport_codes(latitudes, longitudes) # Calling the function

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,53.63040,9.988229
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,48.35380,11.786100


In [42]:
airport_df = icao_airport_codes(latitudes, longitudes)

In [43]:
airport_df

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,53.63040,9.988229
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,48.35380,11.786100


In [44]:
# Creating a copy to keep original Dataframe untouched
airport_df1 = airport_df.copy()
airport_df1

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,53.63040,9.988229
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,48.35380,11.786100


Merging airport with city table

In [45]:
airport_city_merged = airport_df1.merge(city_from_sql, left_on = "municipalityName", right_on = "City", how = "left")

airport_city_merged

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon,City_id,City,Country
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889,1,Berlin,Germany
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,53.63040,9.988229,2,Hamburg,Germany
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,48.35380,11.786100,3,Munich,Germany


In [66]:
airport_city_merged

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon,City_id,City,Country
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889,1,Berlin,Germany
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,53.63040,9.988229,2,Hamburg,Germany
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,48.35380,11.786100,3,Munich,Germany


In [67]:
airport_city_merged_copy = airport_city_merged.copy()

In [68]:
airport_city_merged_copy

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon,City_id,City,Country
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889,1,Berlin,Germany
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,53.63040,9.988229,2,Hamburg,Germany
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,48.35380,11.786100,3,Munich,Germany


In [69]:
airports_df = airport_city_merged_copy = airport_city_merged_copy.drop(columns = ["iata", "shortName", "municipalityName" , "countryCode", "timeZone", "location.lat", "location.lon", "City_id", "City", "Country"])

In [70]:
airports_df

,icao,name
0,EDDB,Berlin Brandenburg
1,EDDH,Hamburg
2,EDDM,Munich


In [71]:
airports_df.rename(columns={"icao": "airport_icao", "name": "airport_name"}, inplace=True)
airports_df

,airport_icao,airport_name
0,EDDB,Berlin Brandenburg
1,EDDH,Hamburg
2,EDDM,Munich


## Pushing the data to airports table in SQL

In [80]:
airports_df.to_sql('airports',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [81]:
airport_df_fromsql = pd.read_sql("airports", con = connection_string)
airport_df_fromsql

,airport_icao,airport_name
0,EDDB,Berlin Brandenburg
1,EDDH,Hamburg
2,EDDM,Munich


In [74]:
airport_city_merged_copy

,airport_icao,airport_name
0,EDDB,Berlin Brandenburg
1,EDDH,Hamburg
2,EDDM,Munich


In [76]:
airport_city_merged_copy2 = airport_city_merged.copy()
airport_city_merged_copy2

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon,City_id,City,Country
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889,1,Berlin,Germany
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,53.63040,9.988229,2,Hamburg,Germany
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,48.35380,11.786100,3,Munich,Germany


In [77]:
cities_airports_df = airport_city_merged_copy2.drop(columns = ["iata", "name", "shortName", "municipalityName" , "countryCode", "timeZone", "location.lat", "location.lon", "City", "Country"])
cities_airports_df

,icao,City_id
0,EDDB,1
1,EDDH,2
2,EDDM,3


In [78]:
cities_airports_df.rename(columns={"icao": "airport_icao"}, inplace=True)
cities_airports_df

,airport_icao,City_id
0,EDDB,1
1,EDDH,2
2,EDDM,3


## Pushing the data to cities_airport table in SQL

In [51]:
airport_city_merged1.rename(columns={"icao": "airport_icao"}, inplace=True)
airport_city_merged1

,airport_icao,City_id
0,EDDB,1
1,EDDH,2
2,EDDM,3


In [52]:
schema = #"your schema" # Name of Schema changes
host = #"host"
user = #"root"
password = #"your password" # Password changes
port = #port number

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [53]:
airport_city_merged1.to_sql('cities_airport',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [113]:
airport_city_merged1_fromsql = pd.read_sql("cities_airport", con = connection_string)
airport_city_merged1_fromsql

,airport_icao,City_id
0,EDDB,1
1,EDDH,2
2,EDDM,3


# Calling the flight data for multiple cities function

Importing the essential libraries

In [96]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone

In [97]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone

def get_flight_data(icao_list):
  api_key = #"your API key"

  berlin_timezone = timezone('Europe/Berlin')
  today = datetime.now(berlin_timezone).date()
  tomorrow = (today + timedelta(days=1))

  flight_items = []

  for icao in icao_list:
    # the api can only make 12 hour calls, therefore, two 12 hour calls make a full day
    # using the nested lists below we can make a morning call and extract the data
    # then make an afternoon call and extract the data
    times = [["00:00","11:59"],
             ["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

      querystring = {"withLeg":"true",
                    "direction":"Arrival",
                    "withCancelled":"false",
                    "withCodeshared":"true",
                    "withCargo":"false",
                    "withPrivate":"false"}

      headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': #api_key
          }

      response = requests.request("GET",
                                  url,
                                  headers = headers,
                                  params = querystring)

      flights_json = response.json()

      retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

      for item in flights_json["arrivals"]:
        flight_item = {
            "arrival_airport_icao": icao,
            "departure_airport_icao": item["departure"]["airport"].get("icao", None),
            "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
            'arrival_terminal': item['arrival'].get('terminal', None),
            "flight_number": item.get("number", None),
            "data_retrieved_at": retrieval_time
        }

        flight_items.append(flight_item)

  flights_df = pd.DataFrame(flight_items)
  flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
  flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
  flights_df["data_retrieved_at"] = pd.to_datetime(flights_df["data_retrieved_at"])

  return flights_df

Calling the function

In [98]:
icao_list = ["EDDB", "EDDH", "EDDM"]

get_flight_data(icao_list)

,arrival_airport_icao,departure_airport_icao,scheduled_arrival_time,arrival_terminal,flight_number,data_retrieved_at
0,EDDB,BIKF,2024-07-09 06:15:00,1,FI 518,2024-07-08 12:54:45
1,EDDB,LTCC,2024-07-09 06:45:00,1,XQ 1700,2024-07-08 12:54:45
2,EDDB,LTBJ,2024-07-09 06:30:00,1,XQ 966,2024-07-08 12:54:45
3,EDDB,None,2024-07-09 07:20:00,1,5F 611,2024-07-08 12:54:45
4,EDDB,EVRA,2024-07-09 07:40:00,1,A3 3219,2024-07-08 12:54:45
...,...,...,...,...,...,...
922,EDDM,LEPA,2024-07-09 23:25:00,1,DE 1131,2024-07-08 12:54:46
923,EDDM,EGLL,2024-07-09 23:15:00,2,LH 2481,2024-07-08 12:54:46
924,EDDM,LPPT,2024-07-09 23:30:00,2,TP 556,2024-07-08 12:54:46
925,EDDM,LPMA,2024-07-09 23:15:00,1,X3 2847,2024-07-08 12:54:46


In [99]:
flights_df = get_flight_data(icao_list)
flights_df

,arrival_airport_icao,departure_airport_icao,scheduled_arrival_time,arrival_terminal,flight_number,data_retrieved_at
0,EDDB,BIKF,2024-07-09 06:15:00,1,FI 518,2024-07-08 12:55:18
1,EDDB,LTCC,2024-07-09 06:45:00,1,XQ 1700,2024-07-08 12:55:18
2,EDDB,LTBJ,2024-07-09 06:30:00,1,XQ 966,2024-07-08 12:55:18
3,EDDB,None,2024-07-09 07:20:00,1,5F 611,2024-07-08 12:55:18
4,EDDB,EVRA,2024-07-09 07:40:00,1,A3 3219,2024-07-08 12:55:18
...,...,...,...,...,...,...
922,EDDM,LEPA,2024-07-09 23:25:00,1,DE 1131,2024-07-08 12:55:20
923,EDDM,EGLL,2024-07-09 23:15:00,2,LH 2481,2024-07-08 12:55:20
924,EDDM,LPPT,2024-07-09 23:30:00,2,TP 556,2024-07-08 12:55:20
925,EDDM,LPMA,2024-07-09 23:15:00,1,X3 2847,2024-07-08 12:55:20


In [100]:
flights_df1 = flights_df.copy()
flights_df1

,arrival_airport_icao,departure_airport_icao,scheduled_arrival_time,arrival_terminal,flight_number,data_retrieved_at
0,EDDB,BIKF,2024-07-09 06:15:00,1,FI 518,2024-07-08 12:55:18
1,EDDB,LTCC,2024-07-09 06:45:00,1,XQ 1700,2024-07-08 12:55:18
2,EDDB,LTBJ,2024-07-09 06:30:00,1,XQ 966,2024-07-08 12:55:18
3,EDDB,None,2024-07-09 07:20:00,1,5F 611,2024-07-08 12:55:18
4,EDDB,EVRA,2024-07-09 07:40:00,1,A3 3219,2024-07-08 12:55:18
...,...,...,...,...,...,...
922,EDDM,LEPA,2024-07-09 23:25:00,1,DE 1131,2024-07-08 12:55:20
923,EDDM,EGLL,2024-07-09 23:15:00,2,LH 2481,2024-07-08 12:55:20
924,EDDM,LPPT,2024-07-09 23:30:00,2,TP 556,2024-07-08 12:55:20
925,EDDM,LPMA,2024-07-09 23:15:00,1,X3 2847,2024-07-08 12:55:20


In [102]:
flights_df1 = flights_df1.drop(columns = ["arrival_terminal", "data_retrieved_at",])

In [103]:
flights_df1

,arrival_airport_icao,departure_airport_icao,scheduled_arrival_time,flight_number
0,EDDB,BIKF,2024-07-09 06:15:00,FI 518
1,EDDB,LTCC,2024-07-09 06:45:00,XQ 1700
2,EDDB,LTBJ,2024-07-09 06:30:00,XQ 966
3,EDDB,None,2024-07-09 07:20:00,5F 611
4,EDDB,EVRA,2024-07-09 07:40:00,A3 3219
...,...,...,...,...
922,EDDM,LEPA,2024-07-09 23:25:00,DE 1131
923,EDDM,EGLL,2024-07-09 23:15:00,LH 2481
924,EDDM,LPPT,2024-07-09 23:30:00,TP 556
925,EDDM,LPMA,2024-07-09 23:15:00,X3 2847


In [104]:
flights_df1.rename(columns={"arrival_airport_icao": "arrival_icao", "departure_airport_icao": "departure_icao"}, inplace=True)
flights_df1

,arrival_icao,departure_icao,scheduled_arrival_time,flight_number
0,EDDB,BIKF,2024-07-09 06:15:00,FI 518
1,EDDB,LTCC,2024-07-09 06:45:00,XQ 1700
2,EDDB,LTBJ,2024-07-09 06:30:00,XQ 966
3,EDDB,None,2024-07-09 07:20:00,5F 611
4,EDDB,EVRA,2024-07-09 07:40:00,A3 3219
...,...,...,...,...
922,EDDM,LEPA,2024-07-09 23:25:00,DE 1131
923,EDDM,EGLL,2024-07-09 23:15:00,LH 2481
924,EDDM,LPPT,2024-07-09 23:30:00,TP 556
925,EDDM,LPMA,2024-07-09 23:15:00,X3 2847


In [107]:
flights_df1['scheduled_arrival_time'] = pd.to_datetime(flights_df1['scheduled_arrival_time'])
flights_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927 entries, 0 to 926
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   arrival_icao            927 non-null    object        
 1   departure_icao          924 non-null    object        
 2   scheduled_arrival_time  927 non-null    datetime64[ns]
 3   flight_number           927 non-null    object        
 4   arrival_time            927 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 36.3+ KB


In [108]:
flights_df1

,arrival_icao,departure_icao,scheduled_arrival_time,flight_number,arrival_time
0,EDDB,BIKF,2024-07-09 06:15:00,FI 518,2024-07-09 06:15:00
1,EDDB,LTCC,2024-07-09 06:45:00,XQ 1700,2024-07-09 06:45:00
2,EDDB,LTBJ,2024-07-09 06:30:00,XQ 966,2024-07-09 06:30:00
3,EDDB,None,2024-07-09 07:20:00,5F 611,2024-07-09 07:20:00
4,EDDB,EVRA,2024-07-09 07:40:00,A3 3219,2024-07-09 07:40:00
...,...,...,...,...,...
922,EDDM,LEPA,2024-07-09 23:25:00,DE 1131,2024-07-09 23:25:00
923,EDDM,EGLL,2024-07-09 23:15:00,LH 2481,2024-07-09 23:15:00
924,EDDM,LPPT,2024-07-09 23:30:00,TP 556,2024-07-09 23:30:00
925,EDDM,LPMA,2024-07-09 23:15:00,X3 2847,2024-07-09 23:15:00


In [109]:
flights_df1 = flights_df1.drop(columns = "arrival_time")

In [110]:
flights_df1

,arrival_icao,departure_icao,scheduled_arrival_time,flight_number
0,EDDB,BIKF,2024-07-09 06:15:00,FI 518
1,EDDB,LTCC,2024-07-09 06:45:00,XQ 1700
2,EDDB,LTBJ,2024-07-09 06:30:00,XQ 966
3,EDDB,None,2024-07-09 07:20:00,5F 611
4,EDDB,EVRA,2024-07-09 07:40:00,A3 3219
...,...,...,...,...
922,EDDM,LEPA,2024-07-09 23:25:00,DE 1131
923,EDDM,EGLL,2024-07-09 23:15:00,LH 2481
924,EDDM,LPPT,2024-07-09 23:30:00,TP 556
925,EDDM,LPMA,2024-07-09 23:15:00,X3 2847


In [111]:
schema = #"your schema" # Name of Schema changes
host = #"host"
user = #"root"
password = #"password" # Password changes
port = # port number

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## Pushing the data to flights table in SQL

In [112]:
flights_df1.to_sql('flights',
                  if_exists='append',
                  con=connection_string,
                  index=False)